# Recommender Systems - notebook 1 - Traditional Approaches

In [2]:
"""
(Practical tip) Table of contents can be compiled directly in jupyter notebooks using the following code:
I set an exception: if the package is in your installation you can import it otherwise you download it 
then import it.
"""
try:
    from jyquickhelper import add_notebook_menu 
except:
    !pip install jyquickhelper
    from jyquickhelper import add_notebook_menu
    
"""
Output Table of contents to navigate easily in the notebook. 
For interested readers, the package also includes Ipython magic commands to go back to this cell
wherever you are in the notebook to look for cells faster
"""
add_notebook_menu()

## Imports

In [3]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
from tqdm import tqdm
import pandas as pd
import numpy as np



## Dataset description

We use here the MovieLens 100K Dataset. It contain 100,000 ratings from 1000 users on 1700 movies.

* u.train / u.test part of the original u.data information
    * The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC   
* u.info
    * The number of users, items, and ratings in the u data set.
* u.item
    * Information about the items (movies); this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.
* u.genre
    * A list of the genres.
* u.user
    * Demographic information about the users; this is a tab
              separated list of
              user id | age | gender | occupation | zip code
              The user ids are the ones used in the u.data data set.

In [5]:
path = "https://www.i3s.unice.fr/~riveill/dataset/dataset_movilens_100K/" 

Before we build our model, it is important to understand the distinction between implicit and explicit feedback, and why modern recommender systems are built on implicit feedback.

* **Explicit Feedback:** in the context of recommender systems, explicit feedback are direct and quantitative data collected from users.
* **Implicit Feedback:** on the other hand, implicit feedback are collected indirectly from user interactions, and they act as a proxy for user preference. 

The advantage of implicit feedback is that it is abundant. Recommender systems built using implicit feedback allow recommendations to be adapted in real time, with each click and interaction.

Today, online recommender systems are built using implicit feedback.

### Data preprocessing

In [6]:
# Load data
np.random.seed(123)

ratings = pd.read_csv(path+'u.data', sep='\t', header=None,
                            names=['userId', 'movieId', 'rating', 'timestamp'])
ratings = ratings.sort_values(['timestamp'], ascending=True)
print("Nb ratings:", len(ratings))
ratings.head()

Nb ratings: 100000


,userId,movieId,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843


### Data splitting

Separating the dataset between train and test in a random fashion would not be fair, as we could potentially use a user's recent evaluations for training and previous evaluations. This introduces a data leakage with an anticipation bias, and the performance of the trained model would not be generalizable to real world performance.

Therefore, we need to slice the train and test based on the timestamp


In [7]:
# Split dataset
train_ratings, test_ratings = np.split(ratings, [int(.9*len(ratings))])

max(train_ratings['timestamp']) <= min(test_ratings['timestamp'])

True

In [8]:
# drop columns that we no longer need
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

len(train_ratings), len(test_ratings)

(90000, 10000)

In [9]:
# Get a list of all movie IDs
all_movieIds = ratings['movieId'].unique()

### Build pivot table

In [10]:
''' Pivot table for train set '''
train_pivot = pd.pivot_table(train_ratings, values='rating', index='userId', columns='movieId')
print("Nb users: ", train_pivot.shape[0])
print("Nb movies:", train_pivot.shape[1])
display(train_pivot.head())

Nb users:  867
Nb movies: 1637


movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train_users = train_pivot.index
train_movies = train_pivot.columns

## Collaborative filtering based on Users similarity

This approach uses scores that have been assigned by other users to calculate predictions.

In pivot table
* Rows are users, $u, v$
* Columns are items, $i, j$

$$pred(u, i) = \frac{\sum_v sim(u,v)*r_{v,i}}{\sum_v sim(u,v)}$$

Wich similarity function:

* Euclidean distance $[0,1]$: $sim(a,b)=\frac{1}{1+\sqrt{\sum_i (r_{a,i}-r_{b,i})^2}}$
* Pearson correlation $[-1,1]$: $sim(a,b)=\frac{\sum_i (r_{a,i}-r_a)(r_{b,i}-r_b)}{{\sum_i (r_{a,i}-r_a)^2}{\sum_i (r_{b,i}-r_b)^2}}$
* Cosine similarity $[-1,1]$: $sim(a, b)=\frac{a.b}{|a|.|b|}$

Which function should we use? The answer is that there is no fixed recipe; but there are some issues we can take into account when choosing the proper similarity function. On the one hand:
* Pearson correlation usually works better than Euclidean distance since it is based more on the ranking than on the values. So, two users who usually like more the same set of items, although their rating is on different scales, will come out as similar users with Pearson correlation but not with Euclidean distance.
* On the other hand, when dealing with binary/unary data, i.e., like versus not like or buy versus not buy, instead of scalar or real data like ratings, cosine distance is usually used.

### Build predictor

In [12]:
# Step 1: build the similarity matrix between users
correlation_matrix = train_pivot.transpose().corr('pearson')
correlation_matrix.head()

userId,1,2,3,5,6,7,8,9,10,12,...,934,935,936,937,938,939,940,941,942,943
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.160841,0.112780,0.420809,0.287159,0.123713,0.692086,-0.102062,-0.092344,0.042191,...,0.061695,-0.260242,0.383733,0.029000,0.326744,0.534390,0.263289,0.205616,-0.180784,0.067549
2,0.160841,1.000000,0.067420,0.327327,0.446269,0.480734,0.585491,0.242536,0.668145,0.000000,...,0.021007,-0.271163,0.214017,0.561645,0.331587,0.000000,-0.011682,-0.062017,0.085960,0.479702
3,0.112780,0.067420,1.000000,NaN,-0.109109,0.000000,0.291937,NaN,0.311086,0.188982,...,NaN,NaN,-0.045162,0.000000,-0.137523,NaN,-0.104678,1.000000,-0.011792,NaN
5,0.420809,0.327327,NaN,1.000000,0.241817,0.149037,0.537400,0.577350,0.087343,0.116981,...,0.229532,-0.500000,0.439286,0.608581,0.484211,0.880705,0.027038,0.468521,0.318163,0.346234
6,0.287159,0.446269,-0.109109,0.241817,1.000000,0.175869,0.687745,0.132353,0.273987,0.112194,...,-0.064033,-0.264292,0.392760,0.270666,0.229918,0.206315,-0.024419,0.399186,0.092349,0.109833


In [19]:
# Step2 build rating function
# We want to calculate the rating that a user could have made for an item.

# Il est plus efficace de travailler avec numpy qu'avec pandas.
# On transforme donc la matrice pivot en numpy
pivot = train_pivot.to_numpy()
# idem pour la matrice de correlation
corr = correlation_matrix.to_numpy()
# Malheureusement, on doit utiliser 2 dictionnaires pour passer
# Du nom de la colonne movieId dans son indice en numpy
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
# Du nom de la ligne userId dans son indice en numpy
user2row = {j:i for i, j in enumerate(train_pivot.index)}

def predict(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u,movie]) and not  np.isnan(corr[user,u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                den += abs(corr[user,u])
                num += corr[user,u] * pivot[u,movie]
        if den != 0:
            return num/den  
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict(pivot, corr, 1, 1)
predict(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


1.8527972301545377

### Predict

In [20]:
# Step 3 add the predicted rating to the test set
pred = []
for _, userId, movieId, _ in tqdm(test_ratings[['userId', 'movieId', 'rating']].itertuples()):
    pred += [predict(pivot, corr, userId, movieId)]

test_ratings['User based'] = pred
test_ratings.head()

10000it [00:11, 894.86it/s]


,userId,movieId,rating,User based
557,90,900,4,0.657995
6675,90,269,5,0.987827
562,90,289,3,0.572064
62660,90,270,4,1.077609
68756,90,268,4,1.202310


### Evaluate the predictor

Now that we have trained our model, assigned a value to each pair (userId, movieId), we are ready to evaluate it.

In [21]:
test_ratings['rating']

557      4
6675     5
562      3
62660    4
68756    4
        ..
46773    4
73008    3
46574    3
64312    4
79208    4
Name: rating, Length: 10000, dtype: int64

#### Evaluation with classical metrics: RMSE

In traditional machine learning projects, we evaluate our models using measures such as accuracy (for classification problems) and RMSE (for regression problems). This is what we will do in the first instance.

In [22]:
# Step 4 evaluate the resulte : with classical metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("RMSE:",
      np.sqrt(mean_squared_error(test_ratings['rating'], test_ratings['User based'])),
     )

RMSE: 1.7798966446725806


#### Hit Ratio @ K

However, a measure such as RMSE does not provide a satisfactory evaluation of recommender systems. To design a good metric for evaluating recommender systems, we need to first understand how modern recommender systems are used.

Amazon, Netflix and others uses a list of recommendations. The key here is that we don’t need the user to interact with every single item in the list of recommendations. Instead, we just need the user to interact with at least one item on the list — as long as the user does that, the recommendations have worked.

To simulate this, let’s run the following evaluation protocol to generate a list of top 10 recommended items for each user.

* For each user, randomly select 99 items that the user has not interacted with.
* Combine these 99 items with the test item (the actual item that the user last interacted with). We now have 100 items.
* Run the model on these 100 items, and rank them according to their predicted probabilities.
* Select the top 10 items from the list of 100 items. If the test item is present within the top 10 items, then we say that this is a hit.
* Repeat the process for all users. The Hit Ratio is then the average hits.

This evaluation protocol is known as **Hit Ratio @ K**, and it is commonly used to evaluate recommender systems.

<font color='red'>
$TO DO - Students$

* Fill the gaps
</font>

In [23]:
# Step 2 with hit ratio
def HRatio(test_ratings, predict, K=10):
    # User-item pairs for testing
    test_user_item_set = set(list(set(zip(test_ratings['userId'], test_ratings['movieId'])))[:1000])

    # Dict of all items that are interacted with by each user
    user_interacted_items = ratings.groupby('userId')["movieId"].apply(list).to_dict()

    hits = []
    for (u,i) in tqdm(test_user_item_set):
        interacted_items = user_interacted_items[u]
        not_interacted_items = set(all_movieIds) - set(interacted_items)
        selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
        test_items = selected_not_interacted + [i]
        predicted_labels = predict([np.array([u]*100), np.array(test_items)]).reshape(-1)
        topK_items = [test_items[i] for i in np.argsort(predicted_labels)[-K:].tolist()]
        
        if i in topK_items:
            hits.append(1)
        else:
            hits.append(0)
    hr = np.average(hits)
    print("The Hit Ratio @ {} is {:.2f}".format(K, hr))
    return hr

In [24]:
def predictor(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict(pivot, corr, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor, 25)
HR['User based'] = hr

100%|██████████| 1000/1000 [01:57<00:00,  8.53it/s]

The Hit Ratio @ 25 is 0.78


## Improve the rating

### Trick 1:

Since humans do not usually act the same as critics, i.e., some people usually rank movies higher or lower than others, this prediction function can be easily improved by taking into account the user mean as follows:

$$pred(u, i) = \overline{r_u} + \frac{\sum_v sim(u,v)*(r_{v,i} - \overline{r_v})}{\sum_v sim(u,v)}$$

<font color='red'>
$TO DO - Students$

* Modify the previous code in order to implement "Trick 1"
</font>

In [25]:
# Step2 build rating function
# We want to calculate the rating that a user could have made for an item.

# Il est plus efficace de travailler avec numpy qu'avec pandas.
# On transforme donc la matrice pivot en numpy
pivot = train_pivot.to_numpy()
# idem pour la matrice de correlation
corr = correlation_matrix.to_numpy()
# Malheureusement, on doit utiliser 2 dictionnaires pour passer
# Du nom de la colonne movieId dans son indice en numpy
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
# Du nom de la ligne userId dans son indice en numpy
user2row = {j:i for i, j in enumerate(train_pivot.index)}

def predict1(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u,movie]) and not  np.isnan(corr[user,u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                den += abs(corr[user,u])
                num += corr[user,u] * (pivot[u,movie]-np.nanmean(pivot[u,:]))
        if (den != 0):
            return (np.nanmean(pivot[user,:])+ num/den ) 
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict1(pivot, corr, 1, 1)
predict1(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


2.9400162942557957

In [26]:
# def predictorNorm(pairs):
#     pred = []
#     for userId, movieId in zip(pairs[0], pairs[1]):
#         pred += [predict(pivot, corr, userId, movieId)]
#     return np.nanmean(pivot[pairs[0],:]) + np.array(pred)
# HR = dict()
# hr = HRatio(test_ratings, predictorNorm, 25)
# HR['User based'] = hr
def predictor1(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict1(pivot, corr, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor1, 25)
HR['User based'] = hr

100%|██████████| 1000/1000 [03:25<00:00,  4.86it/s]

The Hit Ratio @ 25 is 0.80


### Trick 2:

If two users have very few items in common, let us imagine that there is only one, and the rating is the same, the user similarity will be really high; however, the confidence is really small. It's possible to add a ponderation coefficient.

$$newsim(a, b) = sim(a,b) * \frac{min(N, |P_{a,b}|)}{N}$$ 

where $|P_{a,b}|$ is the number of common items shared by user a and user b. The coefficient is $< 1$ if the number of common movies is $< N$ and $1$ otherwise.

In [27]:
# Count the number of common items shared by user 1 and user 2.
pivot = train_pivot.to_numpy()
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
user2row = {j:i for i, j in enumerate(train_pivot.index)}
a = user2row[1]
b = user2row[2]
common = pivot[a,:] + pivot[b,:]
common = np.count_nonzero(~np.isnan(common))
common

18

<font color='red'>
$TO DO - Students$

* Modify the previous code in order to implement "Trick 2"
</font>

In [35]:
def predict_thresh(pivot,corr,userId,movieId,thresh_common_nb=10): #N
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u, movie]) and not np.isnan(corr[user, u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                common = pivot[user] + pivot[u]  # sum of np arrays propagates nan
                common = np.count_nonzero(~np.isnan(common))
                enough_common_coeff = np.min([thresh_common_nb, common]) / thresh_common_nb
                
                num += enough_common_coeff * corr[user, u] * (pivot[u,movie]-np.nanmean(pivot[u,:])) #previous code without np.nanmean(pivot[u,:])  
                den += abs(enough_common_coeff * corr[user, u])

        if den != 0:
            return num / den

        else:
            return 2.5  # default value
    else:
        # le film a déjà été vu
        print(
            f"l'utilisateur {userId} a déjà vu le film {movieId}",
            f"et lui a donné la note de {pivot[user, movie]}",
        )
        return pivot[user, movie]


predict_thresh(pivot=pivot, corr=corr, userId=1, movieId=1)
predict_thresh(pivot=pivot, corr=corr, userId=3, movieId=28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


0.15603823820361443

In [36]:
# %%
for thresh_common_nb in [10, 15, 20, 30, 50]:
    # Step 3 add the predicted rating to the test set

    test_ratings[f"User based_thresh_{thresh_common_nb}"] = [
        predict_thresh(pivot, corr, userId, movieId, thresh_common_nb)
        for _, userId, movieId, _ in tqdm(
            test_ratings[["userId", "movieId", "rating"]].itertuples()
        )
    ]
    # test_ratings

    # Step 4 evaluate the resulte : with classical metrics

    print(
        f"RMSE for N={thresh_common_nb}:",
        np.sqrt(
            mean_squared_error(
                y_true=test_ratings["rating"],
                y_pred=test_ratings[f"User based_thresh_{thresh_common_nb}"],
            )
        ),
    )

10000it [00:57, 174.87it/s]
5it [00:00, 42.27it/s]

RMSE for N=10: 2.3834225675656664


10000it [00:56, 175.92it/s]
5it [00:00, 43.24it/s]

RMSE for N=15: 2.3825050219020794


10000it [00:54, 181.86it/s]
5it [00:00, 48.11it/s]

RMSE for N=20: 2.3818888078652107


10000it [00:53, 185.19it/s]
5it [00:00, 46.12it/s]

RMSE for N=30: 2.3810418082477667


10000it [00:53, 185.50it/s]

RMSE for N=50: 2.379983234252401


In [40]:
def predictor2(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict_thresh(pivot, corr, userId, movieId)]
    return np.array(pred)

HR = dict()
hr = HRatio(test_ratings, predictor2, 25)
HR['User based'] = hr

100%|██████████| 1000/1000 [04:16<00:00,  3.89it/s]

The Hit Ratio @ 25 is 0.79


## To go further

1. Do the same, but with correlation between items. It's Collaborative filtering based on Items similarity. It's also possible to use the 2 previous trick

1.  Use Matrix factorization: decompose R in P, Q at rank k (i.e. if R is a m.n matrix, P is a m.k matrix and Q is a n.k matrix) the reconstruct R with P and Q (i.e. $\hat{R} = P Q^T$)

1. Use Matrix decomposition: do an truncated SVD decomposition in order to obtain U, S and V, build $\hat{R} = U S V^T$

<font color='red'>
$TO DO - Students$

* Choose, implement and evaluate one of the above strategies.
</font>

### Collaborative filtering based on items similarity

In [41]:
""" TRY to predict with item based collaborative filtering """
# Il est plus efficace de travailler avec numpy qu'avec pandas.
# On transforme donc la matrice pivot en numpy
pivot = train_pivot.to_numpy()
# idem pour la matrice de correlation
corr = correlation_matrix.to_numpy()
# Malheureusement, on doit utiliser 2 dictionnaires pour passer
# Du nom de la colonne movieId dans son indice en numpy
movie2column = {j:i for i, j in enumerate(train_pivot.columns)}
# Du nom de la ligne userId dans son indice en numpy
user2row = {j:i for i, j in enumerate(train_pivot.index)}

def predict(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u,movie]) and not  np.isnan(corr[user,u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                den += abs(corr[user,u])
                num += corr[user,u] * pivot[u,movie]
        if den != 0:
            return num/den  
        else:
            return 2.5 # default value
    else:
        # le film a déjà été vu
        print("l'utilisateur", userId, "a déjà vu le film", movieId, "et lui a donné la note de", pivot[user, movie])
        return pivot[user, movie]
        
predict(pivot, corr, 1, 1)
predict(pivot, corr, 3, 28)

l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


1.8527972301545377

In [42]:
""" Evaluate the result """
test_ratings["items_based"] = [
    predict(pivot, corr, userId, movieId)
    for _, userId, movieId, _ in tqdm(
    test_ratings[["movieId", "userId", "rating"]].itertuples() # invert
    )
    ]
test_ratings

23it [00:00, 224.02it/s]

l'utilisateur 270 a déjà vu le film 90 et lui a donné la note de 5.0
l'utilisateur 301 a déjà vu le film 90 et lui a donné la note de 3.0
l'utilisateur 450 a déjà vu le film 650 et lui a donné la note de 4.0
l'utilisateur 59 a déjà vu le film 90 et lui a donné la note de 2.0


104it [00:00, 247.63it/s]

l'utilisateur 506 a déjà vu le film 90 et lui a donné la note de 2.0
l'utilisateur 690 a déjà vu le film 90 et lui a donné la note de 1.0
l'utilisateur 435 a déjà vu le film 90 et lui a donné la note de 4.0
l'utilisateur 194 a déjà vu le film 90 et lui a donné la note de 3.0
l'utilisateur 805 a déjà vu le film 659 et lui a donné la note de 3.0


221it [00:01, 176.71it/s]

...

9974it [00:37, 267.49it/s]

l'utilisateur 97 a déjà vu le film 189 et lui a donné la note de 4.0
l'utilisateur 313 a déjà vu le film 683 et lui a donné la note de 3.0
l'utilisateur 286 a déjà vu le film 683 et lui a donné la note de 5.0
l'utilisateur 264 a déjà vu le film 683 et lui a donné la note de 2.0
l'utilisateur 682 a déjà vu le film 683 et lui a donné la note de 2.0


10000it [00:37, 263.83it/s]

l'utilisateur 22 a déjà vu le film 683 et lui a donné la note de 1.0
l'utilisateur 328 a déjà vu le film 729 et lui a donné la note de 4.0


,userId,movieId,rating,User based,User based_thresh_10,User based_thresh_15,User based_thresh_20,User based_thresh_30,User based_thresh_50,items_based
557,90,900,4,0.657995,0.074150,0.075139,0.075139,0.075139,0.075139,2.288196
6675,90,269,5,0.987827,0.252482,0.249339,0.249339,0.249339,0.249339,2.500000
562,90,289,3,0.572064,-0.086081,-0.095872,-0.095872,-0.095872,-0.095872,1.018354
62660,90,270,4,1.077609,-0.182394,-0.175816,-0.175816,-0.175816,-0.175816,5.000000
68756,90,268,4,1.202310,0.165150,0.165232,0.165232,0.165232,0.165232,2.831497
...,...,...,...,...,...,...,...,...,...,...
46773,729,689,4,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.091913
73008,729,313,3,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,3.036235
46574,729,328,3,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,4.000000
64312,729,748,4,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.800431


### Matrix factorization

Same approach as in the slides. 

<font color='blue'>
Below is a simple algorithm for factoring a matrix.
</font>

In [61]:
# Matrix factorization from scratch
def matrix_factorization(R, K, steps=10, alpha=0.005):
    '''
    R: rating matrix
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    
    # N: num of User
    N = R.shape[0]
    # M: num of Movie
    M = R.shape[1]

    # P: |U| * K (User features matrix)
    P = np.random.rand(N,K)
    # Q: |D| * K (Item features matrix)
    Q = np.random.rand(M,K).T

    for step in tqdm(range(steps)):
        for i in range(N):
            for j in range(M):
                if not np.isnan(R[i][j]):
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        tmp = P[i][k] + alpha * (2 * eij * Q[k][j])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k])
                        tmp = P[i][k]

    return P, Q.T

In [62]:
# We try first on a toy example
# R: rating matrix
import math
R = [
     [5,3,math.nan,1],
     [4,math.nan,math.nan,1],
     [1,1,math.nan,5],
     [1,math.nan,math.nan,4],
     [0,1,5,4],
     [2,1,3,math.nan],
    ]

R = np.array(R)
# Num of Features
K = 3
 
nP, nQ = matrix_factorization(R, K, steps=10)

nR = np.dot(nP, nQ.T)
nR

100%|██████████| 10/10 [00:00<00:00, 833.76it/s]


array([[1.38772738, 1.0231653 , 0.76584089, 1.20772587],
       [1.57484753, 1.05300137, 0.84484023, 1.08430216],
       [0.91762277, 0.71044267, 1.0473353 , 1.78574679],
       [1.35838449, 1.04331211, 1.17025631, 1.98479796],
       [0.61908344, 0.28977816, 0.79746252, 0.9274601 ],
       [1.02144072, 0.65562418, 0.77132318, 1.01705273]])

In [91]:
""" TRY to predict with matrix factorization """
pred= matrix_factorization(pivot, K , steps=10, alpha=0.005)

100%|██████████| 10/10 [00:52<00:00,  5.21s/it]


In [90]:
""" Evaluate the result """
# print("RMSE:",
#       np.sqrt(mean_squared_error(test_ratings['rating'],pred)),
#      )

' Evaluate the result '

## Decomposition using latent factor.

We use SVD decomposition

In [ ]:
pivot = train_pivot.fillna(0).to_numpy()
max_components = min(train_pivot.shape)-1

In [ ]:
from scipy.sparse.linalg import svds

k = 50
assert k<max_components
    
u, s, v_T = svds(pivot, k=k)
nR = u.dot(np.diag(s).dot(v_T))  # output of TruncatedSVD


In [ ]:
""" TRY to predict with SVD decomposition """

In [ ]:
""" Evaluate the result """